## Import requirements

In [1]:
# requirements:

import numpy as np
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

## The required class

1. BanditMachine: A class that represents the bandit machine.
2. EpsilonGreedy: A class that represents the epsilon greedy algorithm. 
3. UCB: A class that represents the upper confidence bound algorithm.


In [2]:
# class BanditMachine:

class BanditMachine:
    def __init__(self, n_arms):
        self.n_arms = n_arms  # number of arms
        
        self.sigma = 1  # reward distribution's standard deviation
        self.miu_list = np.random.normal(0, self.sigma, n_arms)  # initialize the mean of each arm's reward
        self.counts = np.zeros(n_arms, dtype=int)  # record the number of each arm being pulled
        self.values = np.zeros(n_arms)  # record the estimated value of each arm
        
    def update(self, arm, reward):
        """
        update the arm's information
        :param arm: int, index of the arm
        :param reward: float, reward
        """
        self.counts[arm] += 1
        n = self.counts[arm]
        value = self.values[arm]
        self.values[arm] = ((n - 1) / n) * value + (1 / n) * reward # incremental update of the estimated value

    def play(self, arm):
        """
        play the arm and get the reward
        :param arm: int, index of the arm
        :return: float, reward
        """
        if arm < 0 or arm >= self.n_arms:
            raise ValueError("Arm index is out of range.")
        
        # generate the reward
        reward = np.random.normal(self.miu_list[arm], self.sigma)
        # update the arm's information
        self.update(arm, reward)
        return reward


In [3]:
# class EpsilonGreedy:

class EpsilonGreedy:
    def __init__(self, machines, epsilon):
        """
        :param machines: BanditMachine, multi-armed bandit machine object
        :param epsilon: float, probability of randomly selecting an arm
        """
        self.machines = machines
        self.epsilon = epsilon
        self.arms = machines.miu_list
        self.n_arms = len(self.arms)
        self.counts = np.zeros(self.n_arms, dtype=int)  # record the number of each arm being pulled
        self.values = np.zeros(self.n_arms, dtype=float)  # record the estimated value of each arm
        self.total_counts = 0  # record the total number of arms being pulled

    def select_arm(self):
        # if there are arms that have never been pulled, then pull them
        for arm in range(self.n_arms):
            if self.counts[arm] == 0:
                return arm
        
        # with probability epsilon, randomly select an arm, otherwise select the arm with the highest estimated value
        if np.random.rand() < self.epsilon:
            return np.random.randint(self.n_arms)
        else:
            return np.argmax(self.values)

    def update(self, chosen_arm, reward):
        # update the estimated value of the chosen arm
        self.total_counts += 1
        self.counts[chosen_arm] += 1
        n = self.counts[chosen_arm]
        value = self.values[chosen_arm]
        self.values[chosen_arm] = ((n - 1) / n) * value + (1 / n) * reward

    def play(self):
        # choose an arm and pull it
        chosen_arm = self.select_arm()
        reward = self.machines.play(chosen_arm)
        self.update(chosen_arm, reward)
        return chosen_arm, reward


In [4]:
class UCB:
    def __init__(self, machines, c):
        """
        :param arms: list, each element is the mean of the reward distribution of an arm
        :param c: float, a constant that controls the degree of exploration
        """
        self.machines = machines
        self.c = c 
        self.arms = machines.miu_list # the list of arms
        self.n_arms = len(self.arms) # number of arms
        self.counts = np.zeros(self.n_arms, dtype=int)  # record the number of each arm being pulled
        self.values = np.zeros(self.n_arms, dtype=float)  # record the estimated value of each arm
        self.total_counts = 0 # record the total number of arms being pulled

    def select_arm(self):
        # if there are arms that have never been pulled, then pull them
        for i in range(self.n_arms):
            if self.counts[i] == 0:
                return i
        
        
        # return the arm with the highest UCB value
        ucb_values = self.values + self.c * np.sqrt(np.log(self.total_counts) / (self.counts + 1))
        return np.argmax(ucb_values)

    def update(self, chosen_arm, reward):
        # update the estimated value of the chosen arm
        self.total_counts += 1
        self.counts[chosen_arm] += 1
        n = self.counts[chosen_arm]
        value = self.values[chosen_arm]
        self.values[chosen_arm] = ((n - 1) / n) * value + (1 / n) * reward

    def play(self):
        # choose an arm and pull it
        chosen_arm = self.select_arm()
        reward = self.machines.play(chosen_arm)
        self.update(chosen_arm, reward)
        return chosen_arm, reward


## Visualization

In [5]:
# initialize BanditMachine with 10 arms
bandit = BanditMachine(10)

eps_01 = EpsilonGreedy(bandit,0.1)
eps_00 = EpsilonGreedy(bandit,0)
eps_001 = EpsilonGreedy(bandit,0.01)
ucb_2 = UCB(bandit,2)
ucb_1 = UCB(bandit,1)
ucb_05 = UCB(bandit,0.5)
optimal = np.argmax(bandit.miu_list)
step = 1000
runs = 2000

# define the function to test the algorithm
def test_algo(algo,runs,step):
    Q_runs_lst = [[] for m in range(runs)]
    Q_runs_lst_average = [0 for n in range(step)]
    # 2d list, store the best action judgement variable of all runs of all steps, 1 is the correct action, 0 is not the best action
    action_runs_lst = [[] for k in range(runs)]  

    action_runs_lst_average = [0 for j in range(step)]  # record the average best action rate of each step by all runs
    for run_times in range(0, runs):
        k=1
        algo.counts = np.zeros(algo.n_arms, dtype=int)
        algo.values = np.zeros(algo.n_arms, dtype=float)
        algo.total_counts = 0
        while k<=step:
            reward=algo.play()[1]
            action=algo.play()[0]
            Q_runs_lst[run_times].append(reward)
            sum_best_action = 0
            if action==optimal:
                sum_best_action +=1
            action_runs_lst[run_times].append(sum_best_action)
            k += 1
    for i in range(0, step):
        for j in range(0, runs):
            Q_runs_lst_average[i] = Q_runs_lst_average[i] + Q_runs_lst[j][i]
            action_runs_lst_average[i] = action_runs_lst_average[i] + action_runs_lst[j][i]
        Q_runs_lst_average[i] = Q_runs_lst_average[i] / runs
        action_runs_lst_average[i] = action_runs_lst_average[i] / runs
    return Q_runs_lst_average,action_runs_lst_average


In [6]:
# run the algorithm and get the average reward and best action rate of each step
eps_01_reward,eps_01_rate = test_algo(eps_01,runs,step)
eps_00_reward,eps_00_rate = test_algo(eps_00,runs,step)
eps_001_reward,eps_001_rate = test_algo(eps_001,runs,step)
ucb_1_reward,ucb_1_rate = test_algo(ucb_1,runs,step)
ucb_05_reward,ucb_05_rate = test_algo(ucb_05,runs,step)
ucb_2_reward,ucb_2_rate = test_algo(ucb_2,runs,step)


In [ ]:
# plot the average reward and best action rate of each step
plt.figure(1,figsize=(15,6))

# set the format of the y-axis as a percentage
def to_percent(temp, position):
    return '%1.0f'%(100*temp) + '%'

x = range(step)
plt.plot(x,ucb_2_reward,linewidth=0.5,color='r',label="ucb,c=2")
plt.plot(x,eps_01_reward,linewidth=0.5,label="epsilon_greedy,epsilon=0.1")
plt.plot(x,eps_001_reward,linewidth=0.5,color='g',label="epsilon_greedy,epsilon=0.01")
plt.plot(x,eps_00_reward,linewidth=0.5,color='orange',label="epsilon_greedy,epsilon=0")
plt.xlabel('steps')
plt.ylabel('average reward')
plt.legend(loc="best")
plt.figure(2,figsize=(15,6))
plt.plot(x,ucb_2_rate,linewidth=0.5,color='r',label="ucb,c=2")
plt.plot(x,eps_01_rate,linewidth=0.5,label="epsilon_greedy,epsilon=0.1")
plt.plot(x,eps_001_rate,linewidth=0.5,color='g',label="epsilon_greedy,epsilon=0.01")
plt.plot(x,eps_00_rate,linewidth=0.5,color='orange',label="epsilon_greedy,epsilon=0")
plt.xlabel('steps')
plt.ylabel('%Optimal action')
plt.gca().yaxis.set_major_formatter(FuncFormatter(to_percent))
plt.legend(loc="best")
plt.figure(3,figsize=(15,6))
plt.plot(x,ucb_1_reward,linewidth=0.5,label="ucb,c=1")
plt.plot(x,ucb_2_reward,linewidth=0.5,color='r',label="ucb,c=2")
plt.plot(x,ucb_05_reward,linewidth=0.5,color='g',label="ucb,c=0.5")
plt.xlabel('steps')
plt.ylabel('average reward')
plt.legend(loc="best")
plt.figure(4,figsize=(15,6))
plt.plot(x,ucb_1_rate,linewidth=0.5,label="ucb,c=1")
plt.plot(x,ucb_2_rate,linewidth=0.5,color='r',label="ucb,c=2")
plt.plot(x,ucb_05_rate,linewidth=0.5,color='g',label="ucb,c=0.5")
plt.xlabel('steps')
plt.ylabel('%Optimal action')
plt.gca().yaxis.set_major_formatter(FuncFormatter(to_percent))
plt.legend(loc="best")
plt.show()